In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy
from datetime import datetime
import time
import ta

from plot import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from data import db


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2



SYMBOL = "BTC"
START = "2024"
END = "2024"

DB = db.connect_db("database", interval="1h")
data = DB.get_data(SYMBOL, START, END)
#data = data.loc[start:end]


In [ ]:
data.head()

## Import Pipeline

In [24]:
import joblib

df = data.copy()

pipe = joblib.load("fpipeline.pkl")
X = pipe.fit_transform(df)

## Model

In [25]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model.load_model("catboost_model.cbm")

# Utiliser le modèle importé pour faire des prédictions
X['y'] = model.predict(X)
X['proba'] = model.predict_proba(X)[:,1]

In [ ]:
px.histogram(X['proba'])

### Backtest

In [ ]:

data = data.join(X).dropna()
backtest = data.loc['2024-01-13' : '2024-10-13'].copy()


backtest['pred'] = np.where(backtest['proba'] > 0.6, 1,
                            np.where(backtest['proba'] < 0.4,
                                     -1, 0)
                            )

backtest['stage_1_1'] = np.sign(data['stage_1'].diff().fillna(0))
backtest['position'] = backtest['stage_1'] * backtest['pred']

backtest['cum_ret'] = (1 + backtest['ret']).cumprod()
backtest['strategy'] = backtest['ret'] * backtest['position']
backtest['strategy_cum_ret'] = (1 + backtest['strategy']).cumprod()


px.line(backtest[['cum_ret', 'strategy_cum_ret']])

In [ ]:
pd.crosstab(backtest['y'], backtest['stage_1'], normalize=True)

In [ ]:
pd.crosstab(backtest['y'], backtest['stage_1_1'])

In [33]:
c1 = backtest['y'] == 1
c2_1 = backtest['stage_1_1'] == 1
c2_2 = backtest['stage_1_1'] == -1

to_highlight_long = backtest.loc[c1 & c2_1].index.to_list()
to_highlight_short = backtest.loc[c1 & c2_2].index.to_list()

## Visualisation

In [31]:
def compute_relative_sma_gaps(df, sma_windows, col='close'):
    temp_cols = []
    for i, window in enumerate(sma_windows):
        df[f'sma{i+1}'] = df[col].rolling(window=window).mean()
        temp_cols.append(f'sma{i+1}')
        
    for i in range(len(sma_windows) - 1):
        df[f'gap_sma{i+1}_sma{i+2}'] = (df[f'sma{i+1}'] - df[f'sma{i+2}']) * 100 / df[f'sma{i+2}']
    for i in range(1, len(sma_windows)):
        df[f'rel_gap_sma1_sma{i+1}'] = (df['sma1'] - df[f'sma{i+1}']) * 100 / df[f'sma{i+1}']
        df[f"ret_{i}"] = df['close'].pct_change(i)
    return temp_cols  


sma_windows=[3, 6, 12, 18, 24, 72, 24*6]
cols_to_drop = compute_relative_sma_gaps(df=data, sma_windows=sma_windows)

In [ ]:
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, row_heights = [0.7, 0.3], vertical_spacing = 0.01)

plot_candle(fig, col=1, row=1, data=data, symbol=SYMBOL)
add_line(fig, data, feature="sma2", name="sma2", color="black", col=1, row=1)
add_line(fig, data, feature="sma5", name="sma5", col=1, row=1)
add_line(fig, data, feature="sma6", name="sma6", col=1, row=1)
add_line(fig, data, feature="sma7", name="sma7", col=1, row=1)



fig.add_trace(
    go.Scatter(
        x=to_highlight_long,
        y=[data.loc[datetime]['close'] for datetime in to_highlight_long],
        mode='markers',
        marker=dict(color='red', size=10, symbol='circle'),
        name='Points à visualiser'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=to_highlight_short,
        y=[data.loc[datetime]['close'] for datetime in to_highlight_short],
        mode='markers',
        marker=dict(color='blue', size=10, symbol='circle'),
        name='Points à visualiser'
    ),
    row=1, col=1
)

add_line(fig, data, feature="gap_sma5_sma6", name="gap_sma5_sma6", col=1, row=2)
add_line(fig, data, feature="gap_sma6_sma7", name="gap_sma6_sma7", col=1, row=2)
add_line(fig, data, feature="rel_gap_sma2_sma6", name="rel_gap_sma2_sma6", col=1, row=2)


add_hline(fig=fig, y=3, col=1, row=2)
add_hline(fig=fig, y=-3, col=1, row=2)
add_hline(fig=fig, y=0, col=1, row=2)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(height = 1000 , width = 1500,
                          legend = dict(orientation="h",
                                        yanchor="bottom", y=1,
                                        xanchor="right", x=0.7),
                          margin = {'t':0, 'b':0, 'l':10, 'r':0}
                          )
fig.update_layout(hovermode="x")